In [1]:
from urllib.request import urlopen
import pandas as pd
import json
from random import randint

pd.set_option('expand_frame_repr', False) # 列不换行
pd.set_option('display.max_rows', 5000) # 最多显示行数

# 返回数据网址
# 获取代码网址 http://ifzq.gtimg.cn/appstock/app/kline/mkline?param=sz000001,m5,,,100&_var=m5_today&r=0.5643184591626897
# 正常网址 http://stockhtm.finance.qq.com/sstock/ggcx/000001.shtml
# 
# 关键参数sz000001 10 day 16位随机数

stock_code = 'sz000001' #股票代码
k_type = '5' # 5, 15, 30, 60
num = 100 # 获取k线根数, 不超过320根

'''
参数说明：
可返回股票代码，如sz000001
可返回指数，如sh000001
可返回ETF，如sh5105000
'''

'\n参数说明：\n可返回股票代码，如sz000001\n可返回指数，如sh000001\n可返回ETF，如sh5105000\n'

### 创建16位随机数

In [2]:
def _random(n=16):
    """
    创建n位随机整数
    """
    start = 10**(n-1)
    end = (10**n)-1
    return str(randint(start, end))

### 构建网址

In [3]:
url = 'http://ifzq.gtimg.cn/appstock/app/kline/mkline?param=%s,m%s,,,%s&_var=%s_today&r=0.%s'
url = url % (stock_code, k_type, num, k_type, _random())

print(url)

http://ifzq.gtimg.cn/appstock/app/kline/mkline?param=sz000001,m5,,,100&_var=5_today&r=0.7671807826921377


### 抓取数据

In [4]:
content = urlopen(url=url, timeout=15).read().decode()
content = content.split('=', maxsplit=1)[-1]
content = json.loads(content)

print(type(content))
print(content)

<class 'dict'>
{'code': 0, 'msg': '', 'data': {'sz000001': {'qt': {'sz000001': ['51', '平安银行', '000001', '12.40', '12.56', '12.57', '506230', '211240', '294963', '12.40', '906', '12.39', '4680', '12.38', '482', '12.37', '938', '12.36', '2735', '12.41', '843', '12.42', '256', '12.43', '466', '12.44', '584', '12.45', '686', '15:00:04/12.40/5243/S/6502002/14960|14:57:01/12.39/38/B/47082/14836|14:56:57/12.38/607/S/751561/14831|14:56:54/12.39/5/B/6195/14828|14:56:51/12.39/26/B/32214/14826|14:56:48/12.39/24/B/29736/14822', '20190522153004', '-0.16', '-1.27', '12.57', '12.32', '12.40/506230/628437470', '506230', '62844', '0.29', '8.29', '', '12.57', '12.32', '1.99', '2129.11', '2129.13', '0.94', '13.82', '11.30', '0.60', '6906', '12.41', '7.15', '8.58', '', '', '1.27', '62843.75'], 'market': ['2019-05-23 00:36:01|HK_close_未开盘|SH_close_未开盘|SZ_close_未开盘|US_open_交易中|SQ_close_未开盘|DS_close_未开盘|ZS_close_未开盘|NEWSH_close_未开盘|NEWSZ_close_未开盘|NEWHK_close_未开盘|NEWUS_open_交易中|REPO_close_未开盘|UK_close_已收盘'],

### 整理数据

In [5]:
k_data = content['data'][stock_code]['m' + str(k_type)]

df = pd.DataFrame(k_data)
print(df)

                0      1      2      3      4          5   6       7
0    201905081500  12.59  12.60  12.61  12.59   10866.00  {}  0.6328
1    201905090935  12.52  12.51  12.57  12.45   88067.00  {}  5.1290
2    201905090940  12.51  12.48  12.54  12.44   42401.00  {}  2.4694
3    201905090945  12.48  12.45  12.49  12.44   42517.00  {}  2.4762
4    201905090950  12.44  12.38  12.45  12.35   54620.00  {}  3.1811
5    201905090955  12.38  12.36  12.40  12.35   46734.00  {}  2.7218
6    201905091000  12.37  12.30  12.38  12.30   53812.00  {}  3.1340
7    201905091005  12.30  12.27  12.33  12.27   67213.00  {}  3.9145
8    201905091010  12.27  12.28  12.30  12.25   52789.00  {}  3.0744
9    201905091015  12.28  12.28  12.33  12.26   51575.00  {}  3.0037
10   201905091020  12.28  12.25  12.29  12.23   28187.00  {}  1.6416
11   201905091025  12.25  12.22  12.26  12.21   23786.00  {}  1.3853
12   201905091030  12.22  12.19  12.22  12.15   46752.00  {}  2.7228
13   201905091035  12.19  12.23  1

In [7]:
rename_dict ={0:'candle_end_time', 1:'open', 2:'close', 3:'high', 4:'low', 5:'amount'}
# amount 单位为手
df.rename(columns=rename_dict, inplace=True)

print(df)

    candle_end_time   open  close   high    low     amount   6       7
0      201905081500  12.59  12.60  12.61  12.59   10866.00  {}  0.6328
1      201905090935  12.52  12.51  12.57  12.45   88067.00  {}  5.1290
2      201905090940  12.51  12.48  12.54  12.44   42401.00  {}  2.4694
3      201905090945  12.48  12.45  12.49  12.44   42517.00  {}  2.4762
4      201905090950  12.44  12.38  12.45  12.35   54620.00  {}  3.1811
5      201905090955  12.38  12.36  12.40  12.35   46734.00  {}  2.7218
6      201905091000  12.37  12.30  12.38  12.30   53812.00  {}  3.1340
7      201905091005  12.30  12.27  12.33  12.27   67213.00  {}  3.9145
8      201905091010  12.27  12.28  12.30  12.25   52789.00  {}  3.0744
9      201905091015  12.28  12.28  12.33  12.26   51575.00  {}  3.0037
10     201905091020  12.28  12.25  12.29  12.23   28187.00  {}  1.6416
11     201905091025  12.25  12.22  12.26  12.21   23786.00  {}  1.3853
12     201905091030  12.22  12.19  12.22  12.15   46752.00  {}  2.7228
13    

### 将时间变为可读格式

In [8]:
df['candle_end_time'] = df['candle_end_time'].apply(lambda x:'%s-%s-%s %s:%s' % (x[0:4], x[4:6], x[6:8], x[8:10], x[10:12]))
df['candle_end_time'] = pd.to_datetime(df['candle_end_time'])

df = df[['candle_end_time', 'open', 'high', 'low', 'close', 'amount']]

print(df)

        candle_end_time   open   high    low  close     amount
0   2019-05-08 15:00:00  12.59  12.61  12.59  12.60   10866.00
1   2019-05-09 09:35:00  12.52  12.57  12.45  12.51   88067.00
2   2019-05-09 09:40:00  12.51  12.54  12.44  12.48   42401.00
3   2019-05-09 09:45:00  12.48  12.49  12.44  12.45   42517.00
4   2019-05-09 09:50:00  12.44  12.45  12.35  12.38   54620.00
5   2019-05-09 09:55:00  12.38  12.40  12.35  12.36   46734.00
6   2019-05-09 10:00:00  12.37  12.38  12.30  12.30   53812.00
7   2019-05-09 10:05:00  12.30  12.33  12.27  12.27   67213.00
8   2019-05-09 10:10:00  12.27  12.30  12.25  12.28   52789.00
9   2019-05-09 10:15:00  12.28  12.33  12.26  12.28   51575.00
10  2019-05-09 10:20:00  12.28  12.29  12.23  12.25   28187.00
11  2019-05-09 10:25:00  12.25  12.26  12.21  12.22   23786.00
12  2019-05-09 10:30:00  12.22  12.22  12.15  12.19   46752.00
13  2019-05-09 10:35:00  12.19  12.23  12.17  12.23   29057.00
14  2019-05-09 10:40:00  12.23  12.24  12.15  12.16   2

### 备注

退市股票无分钟数据
分钟级别k线不做复权处理